In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='lhG3EnjKNa7EiPdMOPDVXtCeC25B4zti3v_QXhaRlQYN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'coilp-donotdelete-pr-bysdm2fllbzevf'
object_key = 'Crude Oil Prices Daily.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,Date,Price
0,2-Jan-86,25.56
1,3-Jan-86,26.00
2,6-Jan-86,26.53
3,7-Jan-86,25.85
4,8-Jan-86,25.87


In [9]:
import numpy as np 
import pandas as pd 
import datetime
from pylab import rcParams
import matplotlib.pyplot as plt
import warnings
import itertools
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

In [4]:
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
df = df.sort_values('Date')
df = df.groupby('Date')['Price'].sum().reset_index()
df.set_index('Date', inplace=True)

In [6]:
def DfInfo(df_initial):
    # gives some infos on columns types and numer of null values
    tab_info = pd.DataFrame(df_initial.dtypes).T.rename(index={0: 'column type'})
    tab_info = tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0: 'null values (nb)'}))
    tab_info = tab_info.append(pd.DataFrame(df_initial.isnull().sum() / df_initial.shape[0] * 100).T.
                               rename(index={0: 'null values (%)'}))
    return tab_info

In [7]:
DfInfo(df)

,Price
column type,float64
null values (nb),0
null values (%),0.0


In [10]:
scaler = MinMaxScaler(feature_range = (0, 1))
df = scaler.fit_transform(df)

In [11]:
def create_data_set(_data_set, _look_back=1):
    data_x, data_y = [], []
    for i in range(len(_data_set) - _look_back - 1):
        a = _data_set[i:(i + _look_back), 0]
        data_x.append(a)
        data_y.append(_data_set[i + _look_back, 0])
    return np.array(data_x), np.array(data_y)

In [12]:
train_size = int(len(df) * 0.70)
test_size = len(df) - train_size
train, test = df[0:train_size, :], df[train_size:len(df), :]

In [13]:
look_back =10
x_train,y_train,x_test,y_test = [],[],[],[]
x_train,y_train=create_data_set(train,look_back)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test,y_test=create_data_set(test,look_back)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import seaborn as sns

In [15]:
regressor = Sequential()

regressor.add(LSTM(units = 60, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units = 60, return_sequences = True))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.1))

regressor.add(Dense(units = 1))


In [16]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',patience=5)

In [17]:
history =regressor.fit(x_train, y_train, epochs = 20, batch_size = 64,validation_data=(x_test, y_test), callbacks=[reduce_lr],shuffle=False)

Epoch 1/20
102/102 [==============================] - 6s 32ms/step - loss: 0.0038 - val_loss: 0.0373 - lr: 0.0010
Epoch 2/20
102/102 [==============================] - 2s 21ms/step - loss: 0.0080 - val_loss: 0.0025 - lr: 0.0010
Epoch 3/20
102/102 [==============================] - 2s 18ms/step - loss: 0.0018 - val_loss: 6.5726e-04 - lr: 0.0010
Epoch 4/20
102/102 [==============================] - 2s 18ms/step - loss: 9.5692e-04 - val_loss: 0.0012 - lr: 0.0010
Epoch 5/20
102/102 [==============================] - 2s 20ms/step - loss: 8.9645e-04 - val_loss: 7.1630e-04 - lr: 0.0010
Epoch 6/20
102/102 [==============================] - 2s 22ms/step - loss: 0.0012 - val_loss: 0.0024 - lr: 0.0010
Epoch 7/20
102/102 [==============================] - 2s 23ms/step - loss: 0.0010 - val_loss: 8.9252e-04 - lr: 0.0010
Epoch 8/20
102/102 [==============================] - 2s 20ms/step - loss: 0.0013 - val_loss: 0.0025 - lr: 0.0010
Epoch 9/20
102/102 [==============================] - 2s 19ms/step -

In [18]:
regressor.save('model1.h5')

# # Train The Model On IBM

In [19]:
pwd

'/home/wsuser/work'

In [20]:
!tar -zcvf model1.tgz model1.h5

model1.h5


In [17]:
!pip install ibm_watson_machine_learning

In [22]:
from ibm_watson_machine_learning import APIClient

In [23]:
wl = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "FVZnrj27QTkf3jUgGE4z8iyX2VtJwG6K3wL2iav6ycF1"
}

In [24]:
client = APIClient(wl)

In [25]:
def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [26]:
space_uid = guid_from_space_name(client,'new_space')

In [27]:
print('space-id: ',space_uid)

space-id:  52d60f93-4741-4dbf-a6a2-e1066774a419


In [28]:
client.set.default_space(space_uid)

'SUCCESS'

In [30]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_spec_uid = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [30]:
instance_details = client.service_instance.get_api_key()
instance_details

'FVZnrj27QTkf3jUgGE4z8iyX2VtJwG6K3wL2iav6ycF1'

In [31]:
model_details = client.repository.store_model(model='model1.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"C_OIL",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
}
)

In [32]:
published_model_uid = client.repository.get_model_uid(model_details)
published_model_uid

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


'78f60bca-56a5-4bc6-bf42-daff84e70aa4'

In [35]:
import json

model_details = client.repository.get_details('78f60bca-56a5-4bc6-bf42-daff84e70aa4')
print(json.dumps(model_details, indent=2))


{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "acd9c798-6974-5d2f-a657-ce06e986df4d",
      "name": "tensorflow_rt22.1-py3.9"
    },
    "type": "tensorflow_2.7"
  },
  "metadata": {
    "created_at": "2022-11-15T17:35:24.646Z",
    "id": "78f60bca-56a5-4bc6-bf42-daff84e70aa4",
    "modified_at": "2022-11-15T17:35:28.509Z",
    "name": "C_OIL",
    "owner": "IBMid-666002OO3L",
    "resource_key": "17d649b0-6951-47a1-aa0d-12b4f83800a8",
    "space_id": "52d60f93-4741-4dbf-a6a2-e1066774a419"
  },
  "system": {
    "warnings": []
  }
}


In [33]:
models_details = client.repository.list_models()

------------------------------------  -----  ------------------------  --------------
ID                                    NAME   CREATED                   TYPE
78f60bca-56a5-4bc6-bf42-daff84e70aa4  C_OIL  2022-11-15T17:35:24.002Z  tensorflow_2.7
5ceb8e15-e991-4918-9615-78b0d1a78305  C_OIL  2022-11-15T10:43:38.002Z  tensorflow_2.7
------------------------------------  -----  ------------------------  --------------


In [34]:
client.repository.download('78f60bca-56a5-4bc6-bf42-daff84e70aa4','IBM_model.tar.gb')

Successfully saved model content to file: 'IBM_model.tar.gb'


'/home/wsuser/work/IBM_model.tar.gb'